# Sequece to Sequence

- 자연어 처리에서 기존의 일반적인 판별 모델(NLU)은 문장 내 각 단어들을 각 Node가 처리해 하나의 결과를 도출
- Sequence to Sequence는 입력된 Sequence를 이용해 하나의 나열된 Sequence를 출력하는 구조를 만들어 학습을 수행
- Seq2Seq를 이용하면, 자연어 생성 모델 구성이 가능 (기계번역 / 챗봇 / 문장요약 / STT..)
  사진추추추가가

- Seq2Seq는 LSTM기반의 신경망 알고리즘 구조로 크게 2가지 Part
  1. Encoder : Input Sequence를 받아서 문장을 특정 Vector로 변환
  2. Decoder : 새로운 Input Sequence(번역된 문장)을 받아서 Encoder에서 처리된 Contect Vector를 이용해 Output Sequece 생성
     - Context : Encoder에 의해 변환된 Vector
     - Encoder의 Input과 Decoder의 Input을 Context Vector로 잘 매칭(처리)하여 적절한 Output 도출

- 작동 순서
  1. 입력 문장을 토큰화를 통해 단어(또는 형태소 단위로 처리)
  2. 단어 토큰을 각각 RNN계열의 Node에 입력
  3. Encoder 마지막에 계산된 Node의 정보를 Context Vector로 Decoder에 전달
  4. Decoder는 앞서 입력받은 Context Vector를 첫 Node에 받아서 처리
  5. Decoder에 문장의 첫번째를 알리는 초기 토큰 (Start of Sequence, SOS)을 첫 Node에 입력
  6. 이후, Decoder에서 문장을 이어받아 처리하면서, 다음 등장할 확률이 높은 단어를 예측해 Output
  7. EOS (End of Sequence)값을 Input Sequence 끝에 넣어, 문자으이 마지막을 학습
  8. Output값이 생성 될 때, EOS 토큰이 등장하거나 사용자가 지정한 문장의 최대 길이 만큼 Output 값을 출력 후 생성을 종료
 

In [2]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
import numpy as np

In [5]:
data_path =  r'C:\Users\UserK\Desktop\Ranee\data\ML\kor.txt' 
with open(data_path, encoding = "UTF-8") as file :
    lines = file.read().split('\n') # 데이터 파일을 불러올 때, 띄워쓰기를 기준으로 구분

In [15]:
# Encoder와 Decoder에 입력할때 데이터의 구조를 생성
# 각 문장을 넣을 리스트 구조를 선언
input_text_list = [] # 영어 문장을 리스트로 선언
target_text_list = [] # 한국어 문장을 리스트로 선언
input_characters = set() # 영어 단어 리스트 (중복단어 제거된 단어 사전)
target_characters = set() # 한국어 단어 리스트 (중복 단어가 제거된 단어 사전)

In [40]:
num_samples = len(lines) # 처리할 문장의 수를 변수로 선언
for line in lines[0:num_samples-2]:
    # 문장 처리 단계에서 오류 방지를 위한 Try / Except 구문을 활용
    try:
        # tab key를 기준으로 각 문장을 3개의 토큰으로 분할
        input_text, target_text,_ = line.split('\t')
        # Decoder에 들어갈 Inpit Seq(Target Text)에 SOS토큰과 EOS토큰을 부착
        target_text = '\t' + target_text + '\n' # tab key(SOS)로 Enter Key를 (EOS)로 지정하여 부착
        # 각각 나눠진 Text를 각 리스트에 추가
        input_text_list.append(input_text)
        target_text_list.append(target_text)
    except ValueError: # 문장 처리 중 오류가 발생 했을 시
        print('Skip Line', line) # 어떤 문장이 생략됬는지 확인
        continue # 이어서 진행
    # 분할 된 모든 문장에서 고유 단어만 뽑아 리스트로 선언(사전 만들기 위한 작업 ) 
    for char in input_text:
        input_characters.add(char)
    for char in target_text:
        target_characters.add(char)

In [23]:
# 문자 사전을 만들기 위한 리스트 변환
input_char_list = sorted(list(input_characters)) # 영어 문자 토큰을 리스트로 변환 후 정렬
target_char_list = sorted(list(target_characters)) # 한국어 문자 토큰을 리스트로 변환 후 정렬

# 전체 단어 고유 수 만큼 부여하여. 문자 -> 숫자 사전을 구축
num_encoder_token = len(input_char_list) # encoder에 들어갈 영어 문자 수 
num_decoder_token = len(target_char_list) # decoder에 들어갈 한국어 문자 수

# 영어 문자를 숫자로 변환시킬 Dictionary 구성
input_token_index = dict([(char,i) for i , char in enumerate(input_char_list)])
# 한국어 문자를 숫자로 변환시킬 Dictionary 구성
target_token_index = dict([(char,i) for i , char in enumerate(target_char_list)])

In [24]:
# 전체 문장들 중 최대 길이를 확인
max_encoder_seq_length = max([len(x) for x in input_text_list])
max_decoder_seq_length = max([len(x) for x in target_text_list])

In [27]:
# 각 문장을 변환 된 정수로 집어넣어 Matrix를 구성
encoder_input_data = np.zeros((len(input_text_list), max_encoder_seq_length, num_encoder_token), dtype='float32')
decoder_input_data = np.zeros((len(target_text_list), max_decoder_seq_length, num_decoder_token), dtype='float32')
decoder_target_data = np.zeros((len(target_text_list), max_decoder_seq_length, num_decoder_token), dtype='float32')

In [32]:
# One Hot Encoding
# 구성된 Matrix에 각 단어에 매칭되는 숫자를 입력
# 영어 문장과 한국어 문장을 각각 가져와 동시에 숫자를 Matrix에 입력
for i, (input_text, target_text) in enumerate(zip(input_text_list, target_text_list)) :
    # 영어 문장에 해당하는 단어를 숫자로 변환하여 Maxtrix에 추가
    for t, char in enumerate(input_text) :
        encoder_input_data[i, t, input_token_index[char]] = 1
    # 한국어 문장에 해당하는 단어를 숫자로 변환하여 Maxtrix에 추가
    for t, char in enumerate(target_text) :
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0 :
            decoder_target_data[i, t-1, target_token_index[char]] = 1

In [41]:
# 학습 시간과 컴퓨터 성능 문제를 고려해 데이터를 간소화 하여 학습을 진행
batch_size = 32 # 훈련 데이터 셋 배치 크기(64이상)
epochs = 20 # 훈련 에포크 수(500회 이상)
node_num = 64 # 각 Layer에 들어갈 Node수(1024개 이상)

In [42]:
# 인코더 구성
# 영어 문장이 들어가 LSTM 모델에 의해 학습이 수행

encoder_input = Input(shape=(None, num_encoder_token))
encoder = LSTM(node_num, return_state = True)

# state_h : 마지막 Layer의 State / state_c 마지막 Cell State
encoder_ouput , state_h , state_c = encoder(encoder_input)

# 인코더가 입력 시퀀스를 처리한 후, 얻은 정보를 encoder_state에 선언
# 디코더가 이를 기반으로 출력 시퀀스를 생성 (Context Vector)
encoder_state = [state_h, state_c]

In [43]:
# 디코더 구성
decoder_input = Input(shape=(None, num_decoder_token))

# return_sequence=True  : 각 단계에서 Seq의 전체 출력을 계산하도록 설정
decoder_lstm = LSTM(node_num, return_sequences=True, return_state=True)

# 앞서 처리된 Encoder의 정보를 불러와 Decoder의 초기 상태로 사용
decoder_output, _, _ = decoder_lstm(decoder_input, initial_state = encoder_state)

# Softmax의 확률 분포 값을 이용하여 출력값의 확률 값을 계산
decoder_dense = Dense(num_decoder_token, activation='softmax')
decoder_output = decoder_dense(decoder_output)

# Seq2Seq 모델 정의
# 두개의 Input이 들어가, 하나의 Output 나오는 구조
model = Model( [encoder_input, decoder_input] , decoder_output)

In [45]:
# 학습 실시
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit( [encoder_input_data, decoder_input_data] , decoder_target_data,
          batch_size = batch_size, epochs=epochs, validation_split=0.3)

Epoch 1/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 16s 102ms/step - loss: 0.6232 - val_loss: 1.0328
Epoch 2/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - loss: 0.6066 - val_loss: 1.0244
Epoch 3/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 13s 97ms/step - loss: 0.6024 - val_loss: 1.0198
Epoch 4/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - loss: 0.5951 - val_loss: 1.0085
Epoch 5/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 13s 97ms/step - loss: 0.5886 - val_loss: 0.9907
Epoch 6/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 13s 104ms/step - loss: 0.5780 - val_loss: 0.9818
Epoch 7/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step - loss: 0.5705 - val_loss: 0.9717
Epoch 8/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step - loss: 0.5726 - val_loss: 0.9700
Epoch 9/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step - loss: 0.5672 - val_loss: 0.9675
Epoch 10/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - loss: 0.5633 - val_loss: 0.9625
Epoch 11/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - loss: 0.5629 - val_loss: 0.9613
Epoch 12/20
129/129 ━━━━━━━━

In [46]:
# 학습 이후(NLU) 추론 및 생성(NLG) 모델을 구성
# 학습 과정에서 구성한 인코더의 입력과 인코더의 상태를 기반으로 새로운 모델을 생성
encoder_model = Model(encoder_input, encoder_state)

# 디코더의 LSTM 레이어에 전달된 초기 상태를 정의하는 입력 Layer를 구성
decoder_state_input_h = Input(shape=(node_num, )) # Layer State를 초기화
decoder_state_input_c = Input(shape=(node_num, )) # Cell State를 초기화

# 초기 Decoder의 Input Node를 구성
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]

# 구성된 Input Node를 이용해, Decoder의 Layer를 구성
decoder_output, state_h, state_c = decoder_lstm(decoder_input, initial_state= decoder_state_input)

# 다음 Step으로 전달 될 상태를 지정
decoder_output = [state_h, state_c]

# 확률 분포를 계산하는 Decoder Output에 전달 하여 Update
decoder_output_proba = decoder_dense(decoder_output)

# 다음 단계의 토큰(단어,문자)를 예측하고 다음 문장을 생성하기 위한 생태를 생성
decoder_model = Model([decoder_input] + decoder_state_input,
                      [decoder_output]+decoder_state_)


# Decoder가 계산 결과 이용해, 역매핑 (숫자->문자) 변환 작업으르 수행
reverse_input_char_index = dict( (i,char) for char,i in input_token_index.items())
reverse_target_char_index = dict( (i,char) for char,i in target_token_index.items())

ValueError: Layer 'dense_2' expected 1 input(s). Received 2 instead.

In [ ]:
# 입력 문장이 들어오면, 해당 문장을 번역하는 함수를 구성
def decode_sequence(input_seq) :
    # 입력 받은 Sequence를 이용해, Encoder에 넣어 Context Vector를 예측
    state_value = encoder_model.predict(input_seq, verbose = 0 )

In [ ]:
def preprocess_input_sequence(sentence) :
    input_seq = np.zeros((1,max_encoder_seq_length, num_encoder_token), dtype='float32')